In [ ]:
import torch
import numpy as np
import non_local_boxes
import matplotlib.pyplot as plt
from matplotlib.colors import hsv_to_rgb
import matplotlib.patches as mpatches

# Sugar coating for reloading
%matplotlib inline
%load_ext autoreload
%autoreload 2

# Implementation of `rho_EG16(P)`

Based on [EG16]: https://ieeexplore.ieee.org/abstract/document/7329921?casa_token=Gru1-7wNm_EAAAAA:0zLAco1Mrd-RUQYIwCOTYEL6gASgAOIHAFyr23nqnLC-41DyYOAQ9zD3LxbPbHwoiFDaDQ-PSaee

In [ ]:
def rho_EG16_max(P):   # P is a 2x2x2x2 tensor
    values = np.zeros((2,2))
    for x in range(2):
        for y in range(2):
            prob_a_is_0 = P[0, 0, x, y] + P[0, 1, x, y]
            prob_a_is_1 = P[1, 0, x, y] + P[1, 1, x, y]
            prob_b_is_0 = P[0, 0, x, y] + P[1, 0, x, y]
            prob_b_is_1 = P[0, 1, x, y] + P[1, 1, x, y]
            mu_A = prob_a_is_1/prob_a_is_0
            mu_B = prob_b_is_1/prob_b_is_0
            f1 = 1/np.sqrt((1+mu_A)*prob_a_is_1)
            f0 = -f1 * mu_A
            g1 = 1/np.sqrt((1+mu_B)*prob_b_is_1)
            g0 = -g1 * mu_B
            values[x,y] = f0 * g0 * P[0,0,x,y] + f1 * g0 * P[1,0,x,y] + f0 * g1 * P[0,1,x,y] + f1 * g1 * P[1,1,x,y]
    return np.max(values)

In [ ]:
def rho_EG16_average(P):   # P is a 2x2x2x2 tensor
    values = np.zeros((2,2))
    for x in range(2):
        for y in range(2):
            prob_a_is_0 = P[0, 0, x, y] + P[0, 1, x, y]
            prob_a_is_1 = P[1, 0, x, y] + P[1, 1, x, y]
            prob_b_is_0 = P[0, 0, x, y] + P[1, 0, x, y]
            prob_b_is_1 = P[0, 1, x, y] + P[1, 1, x, y]
            mu_A = prob_a_is_1/prob_a_is_0
            mu_B = prob_b_is_1/prob_b_is_0
            f1 = 1/np.sqrt((1+mu_A)*prob_a_is_1)
            f0 = -f1 * mu_A
            g1 = 1/np.sqrt((1+mu_B)*prob_b_is_1)
            g0 = -g1 * mu_B
            values[x,y] = f0 * g0 * P[0,0,x,y] + f1 * g0 * P[1,0,x,y] + f0 * g1 * P[0,1,x,y] + f1 * g1 * P[1,1,x,y]
    return np.average(values)

In [ ]:
def draw_triangle(Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size, function_rho, function_name): # Box is a 4x4 tensor
    X = torch.tensor([[0, 10], [10,0], [0, 0]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)

    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            p = function_rho(alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3)
            color_point = hsv_to_rgb([p, 1, 1]) #(0., 0.*(1-p)+0.8*p, 0.*(1-p)+0.8*p)
            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)

    plt.text(X[0,0]+0.3, X[0,1], Box1_name, horizontalalignment='center')
    plt.text(X[1,0], X[1,1]+0.3, Box2_name, verticalalignment='center')
    plt.text(X[2,0], X[2,1]-0.3, Box3_name, verticalalignment='center')
    plt.title("Level lines of "+function_name)
    color1 = mpatches.Patch(color=hsv_to_rgb([1, 1, 1]), label='rho = 1')
    color075 = mpatches.Patch(color=hsv_to_rgb([0.75, 1, 1]), label='rho = 0.75')
    color05 = mpatches.Patch(color=hsv_to_rgb([0.5, 1, 1]), label='rho = 0.5')
    color025 = mpatches.Patch(color=hsv_to_rgb([0.25, 1, 1]), label='rho = 0.25')
    plt.legend(handles=[color1, color075, color05, color025]) 
    plt.show()

In [ ]:
def draw_above_BBLMTU(Box1, Box1_name, Box2, Box2_name, Box3, Box3_name, box_grid_size, function_rho, function_name): # Box is a 4x4 tensor
    X = torch.tensor([[0, 10], [10,0], [0, 0]]).numpy()
    triangle = plt.Polygon(X[:3,:], color="snow")
    plt.gca().add_patch(triangle)
    a = (3 + float(torch.sqrt(torch.tensor(6))))/6
    threshold = function_rho(a*non_local_boxes.utils.matrix_to_tensor(non_local_boxes.utils.PR) + (1-a)*non_local_boxes.utils.matrix_to_tensor(non_local_boxes.utils.PRbar))

    color_above_threshold = hsv_to_rgb([0.475, 1, 0.8])
    color_below_threshold = hsv_to_rgb([0.475, 1, 0.5])

    for i in range(box_grid_size+1):
        for j in range(box_grid_size-i+1):
            alpha, beta = i/box_grid_size, j/box_grid_size
            p = function_rho(alpha*Box1 + beta*Box2 + (1-alpha-beta)*Box3)
            color_point = color_below_threshold #(0., 0.3, 0.3)
            if p > threshold:
                color_point = color_above_threshold # (0., 0.7, 0.7)
            plt.plot(X[0,0]*alpha + X[1,0]*beta + X[2,0]*(1-alpha-beta), X[0,1]*alpha + X[1,1]*beta + X[2,1]*(1-alpha-beta), 'o', markersize=3, color=color_point)

    plt.text(X[0,0]+0.3, X[0,1], Box1_name, horizontalalignment='center')
    plt.text(X[1,0], X[1,1]+0.3, Box2_name, verticalalignment='center')
    plt.text(X[2,0], X[2,1]-0.3, Box3_name, verticalalignment='center')
    plt.title(function_name+" combined with [BBLMTU06]")
    color_below_patch = mpatches.Patch(color=color_below_threshold, label='Impossible to distill until the collapsing triangle.')
    color_above_patch = mpatches.Patch(color=color_above_threshold, label='No conclusion.')
    plt.legend(handles=[color_below_patch, color_above_patch])   
    plt.show()

# Compute $\rho$ of somes boxes

In [ ]:
PR = non_local_boxes.matrix_to_tensor(non_local_boxes.utils.PR)
PRprime = non_local_boxes.matrix_to_tensor(non_local_boxes.utils.PRprime)
SR = non_local_boxes.matrix_to_tensor(non_local_boxes.utils.SR)
I = non_local_boxes.matrix_to_tensor(non_local_boxes.utils.I)

In [ ]:
p = 0.156
print(rho_EG16_max(PR), rho_EG16_max(SR), rho_EG16_max(I), rho_EG16_max((PR+I)/2))
print(rho_EG16_max(p*PR + (1-p)*I))

In [ ]:
draw_triangle(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "SR",
    Box3 = (PR+I)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_max, 
    function_name = "rho_EG16_max"
)

In [ ]:
draw_above_BBLMTU(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "SR",
    Box3 = (PR+I)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_max, 
    function_name = "rho_EG16_max"
)

In [ ]:
draw_above_BBLMTU(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "(PR+PRpime)/2",
    Box3 = (PR+PRprime)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_max, 
    function_name = "rho_EG16_max"
)

# $\rho$ average

In [ ]:
draw_triangle(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "SR",
    Box3 = (PR+I)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_average, 
    function_name = "rho_EG16_average"
)

In [ ]:
draw_above_BBLMTU(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "SR",
    Box3 = (PR+I)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_average, 
    function_name = "rho_EG16_average"
)

In [ ]:
draw_above_BBLMTU(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "(PR+PRpime)/2",
    Box3 = (PR+PRprime)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_average, 
    function_name = "rho_EG16_average"
)

# $\rho$ CHSH

In [ ]:
def rho_EG16_CHSH(P):   # P is a 2x2x2x2 tensor
    values = np.zeros((2,2))
    for x in range(2):
        for y in range(2):
            prob_a_is_0 = P[0, 0, x, y] + P[0, 1, x, y]
            prob_a_is_1 = P[1, 0, x, y] + P[1, 1, x, y]
            prob_b_is_0 = P[0, 0, x, y] + P[1, 0, x, y]
            prob_b_is_1 = P[0, 1, x, y] + P[1, 1, x, y]
            mu_A = prob_a_is_1/prob_a_is_0
            mu_B = prob_b_is_1/prob_b_is_0
            f1 = 1/np.sqrt((1+mu_A)*prob_a_is_1)
            f0 = -f1 * mu_A
            g1 = 1/np.sqrt((1+mu_B)*prob_b_is_1)
            g0 = -g1 * mu_B
            values[x,y] = f0 * g0 * P[0,0,x,y] + f1 * g0 * P[1,0,x,y] + f0 * g1 * P[0,1,x,y] + f1 * g1 * P[1,1,x,y]
    return (values[0,0]+values[0,1]+values[1,0]-values[1,1])/4

In [ ]:
draw_triangle(
    Box1 = PR,
    Box1_name = "PR",
    Box2 = SR,
    Box2_name = "SR",
    Box3 = (PR+I)/2,
    Box3_name = "(PR+I)/2",
    box_grid_size = 60,
    function_rho = rho_EG16_CHSH, 
    function_name = "rho_EG16_CHSH"
)